In [1]:
import pandas as pd
import numpy as np

## Importando Peças Coletadas

In [2]:
dfColeta = pd.read_csv('input/coleta.txt', delimiter = ' ').rename(columns = {'Peças': 'Peças Coletadas'})
dfColeta = dfColeta[dfColeta["Peças Coletadas"].astype('string').str.len() == 16]
dfColeta

,Peças Coletadas
0,1000972458101201
1,1000972457101202
2,1000972456101203
3,1000972459101204
4,1000972460101205
...,...
177,1000207331536104
178,1000192665536105
179,1000902827538101
180,1000913714538102


In [3]:
dfVA = pd.read_csv('input/peçasDoSistema.CSV', delimiter = ';', usecols = ["Codigo 1000", "Local"], dtype = {"Codigo 1000": np.int64}).rename(columns = {'Codigo 1000' : 'Peças VA', 'Local' : 'Local VA'})[["Peças VA", "Local VA"]]
dfVA

,Peças VA,Local VA
0,1000989323,RUA A COLUNA 02 GAVETA 03
1,1000989319,RUA A COLUNA 02 GAVETA 03
2,1000989320,RUA A COLUNA 02 GAVETA 03
3,1000893414,RUA A COLUNA 02 GAVETA 06
4,1000993093,RUA A COLUNA 02 GAVETA 04
...,...,...
62,1000923389,RUA A COLUNA 02 GAVETA 07
63,1000923388,RUA A COLUNA 02 GAVETA 07
64,1000923386,RUA A COLUNA 02 GAVETA 07
65,1000923385,RUA A COLUNA 02 GAVETA 07


-----
## Tratamento CSV
- Peças com Local Diferente
- Peças Somente no Sistema
- Peças Somente na Coleta

In [16]:
def dataFrameToExcel_PcLD(df, endereco):
    
    condition = ((df["Local Coletado"].notna()) & (df["Local VA"].notna())) & (df["Local Coletado"] != df["Local VA"])
    
    filename = "Peças com Local Diferente.xlsx"
    
    df[condition].to_excel((endereco + filename), index=False)
    
def dataFrameToExcel_PSnS(df, endereco):
    
    filename = "Peças Somente no Sistema.xlsx"
    
    df[df["Local Coletado"].isna()][["Peça", "Local VA"]].to_excel((endereco + filename), index=False)
    
def dataFrameToExcel_PSnC(df, endereco):
    
    filename = "Peças Somente na Coleta.xlsx"
    
    df[df["Local VA"].isna()][["Peça", "Local Coletado"]].to_excel((endereco + filename), index=False)
    

def dataFrameToExcel(df, endereço):
    dataFrameToExcel_PcLD(df, endereço)
    dataFrameToExcel_PSnS(df, endereço)
    dataFrameToExcel_PSnC(df, endereço)

## Função de Transformação - DataFrame em Excel

In [5]:
def dataFrameToExcel(df, endereco):
    df.to_excel(endereco, index=False)

## Funções de Transformação - Local Coletado para Local VA

In [6]:
def defineRua(string):
    
    if string == '1':
        return 'A'
    elif string == '2':
        return 'B'
    elif string == '3':
        return 'C'
    elif string == '4':
        return 'D'
    elif string == '5':
        return 'E'
    elif string == '6':
        return 'F'
    elif string == '7':
        return 'G'
    elif string == '8':
        return 'H'
    elif string == '9':
        return 'I'
    else:
        return "ERRO"

def defineColunaEGaveta(n):
    
    col = n[0:2]
    gav = n[2]

    return " COLUNA {} GAVETA 0{}".format(col, gav)

def transformaLocal(df, coluna):
    
    df[coluna] = df[coluna].map(lambda x: "RUA " + defineRua(str(x)[0]) + defineColunaEGaveta(str(x)[1:4]))

## Função de Transformação - Formato coleta para formato Virtual Age

In [7]:
def coletaParaVA(df):
    df.insert(1, "Local Coletado", ['' for i in range(0, len(df.index))], allow_duplicates = True)
    
    df["Local Coletado"] = df["Peças Coletadas"].map(lambda x: int(str(x)[-6:]))
    transformaLocal(df, "Local Coletado")
    df["Peças Coletadas"] = df["Peças Coletadas"].map(lambda x: int(str(x)[:-6]))

    return df

## Função de União - União entre os 2 DataFrames

In [8]:
def uniaoDF(df1, df2):
    
    dfFinal = df1
    
    dfFinal.insert(2, "Peças Coletadas", df2["Peças Coletadas"], allow_duplicates = True)
    dfFinal.insert(3, "Local Coletado", df2["Local Coletado"], allow_duplicates = True)
    
    return dfFinal

# Execução do Teste

In [9]:
dfColeta = coletaParaVA(dfColeta)
dfColeta

,Peças Coletadas,Local Coletado
0,1000972458,RUA A COLUNA 01 GAVETA 02
1,1000972457,RUA A COLUNA 01 GAVETA 02
2,1000972456,RUA A COLUNA 01 GAVETA 02
3,1000972459,RUA A COLUNA 01 GAVETA 02
4,1000972460,RUA A COLUNA 01 GAVETA 02
...,...,...
177,1000207331,RUA E COLUNA 36 GAVETA 01
178,1000192665,RUA E COLUNA 36 GAVETA 01
179,1000902827,RUA E COLUNA 38 GAVETA 01
180,1000913714,RUA E COLUNA 38 GAVETA 01


In [10]:
dfVA = dfVA.rename(columns = {"Peças VA" : "Peça"})
dfColeta = dfColeta.rename(columns = {"Peças Coletadas" : "Peça"})

dfMergeVA = dfVA.merge(dfColeta, how = "left", on = "Peça")
dfMergeColeta = dfColeta.merge(dfVA, how = "left", on = "Peça")

dfMergeVA

,Peça,Local VA,Local Coletado
0,1000989323,RUA A COLUNA 02 GAVETA 03,NaN
1,1000989319,RUA A COLUNA 02 GAVETA 03,NaN
2,1000989320,RUA A COLUNA 02 GAVETA 03,NaN
3,1000893414,RUA A COLUNA 02 GAVETA 06,NaN
4,1000993093,RUA A COLUNA 02 GAVETA 04,NaN
...,...,...,...
62,1000923389,RUA A COLUNA 02 GAVETA 07,NaN
63,1000923388,RUA A COLUNA 02 GAVETA 07,NaN
64,1000923386,RUA A COLUNA 02 GAVETA 07,NaN
65,1000923385,RUA A COLUNA 02 GAVETA 07,NaN


In [11]:
dfMergeColeta

,Peça,Local Coletado,Local VA
0,1000972458,RUA A COLUNA 01 GAVETA 02,NaN
1,1000972457,RUA A COLUNA 01 GAVETA 02,NaN
2,1000972456,RUA A COLUNA 01 GAVETA 02,NaN
3,1000972459,RUA A COLUNA 01 GAVETA 02,NaN
4,1000972460,RUA A COLUNA 01 GAVETA 02,NaN
...,...,...,...
177,1000207331,RUA E COLUNA 36 GAVETA 01,NaN
178,1000192665,RUA E COLUNA 36 GAVETA 01,NaN
179,1000902827,RUA E COLUNA 38 GAVETA 01,NaN
180,1000913714,RUA E COLUNA 38 GAVETA 01,NaN


In [12]:
mergeAll = pd.concat([dfMergeVA, dfMergeColeta], ignore_index=True)
mergeAll

,Peça,Local VA,Local Coletado
0,1000989323,RUA A COLUNA 02 GAVETA 03,NaN
1,1000989319,RUA A COLUNA 02 GAVETA 03,NaN
2,1000989320,RUA A COLUNA 02 GAVETA 03,NaN
3,1000893414,RUA A COLUNA 02 GAVETA 06,NaN
4,1000993093,RUA A COLUNA 02 GAVETA 04,NaN
...,...,...,...
244,1000207331,NaN,RUA E COLUNA 36 GAVETA 01
245,1000192665,NaN,RUA E COLUNA 36 GAVETA 01
246,1000902827,NaN,RUA E COLUNA 38 GAVETA 01
247,1000913714,NaN,RUA E COLUNA 38 GAVETA 01


In [17]:
dataFrameToExcel(mergeAll, "output/")